This notebook requires version 0.14.0 of the `visual_behavior_analysis` package

And version 2.16.2 of the `AllenSDK`

In [2]:
import os
import numpy as np
import pandas as pd

from visual_behavior.data_access import utilities
import visual_behavior.visualization.utils as utils
import visual_behavior.data_access.loading as loading

import seaborn as sns
sns.set_context('notebook', font_scale=1.5, rc={'lines.markeredgewidth': 1})

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

#### Get metadata for each experiment (2-photon imaging plane)

In [4]:
# set cache directory where files should be loaded from and saved to
platform_cache_dir = loading.get_platform_analysis_cache_dir()
print(platform_cache_dir)

\\allen\programs\braintv\workgroups\nc-ophys\visual_behavior\platform_paper_cache_new


In [5]:
from allensdk.brain_observatory.behavior.behavior_project_cache import VisualBehaviorOphysProjectCache

cache = VisualBehaviorOphysProjectCache.from_s3_cache(cache_dir = platform_cache_dir)

# load full ophys experiments table from the cache
all_ophys_experiments_table = cache.get_ophys_experiment_table()
print(len(all_ophys_experiments_table))

1936


C:\Users\marinag\Documents\Code\AllenSDK\allensdk\brain_observatory\behavior\behavior_project_cache\behavior_project_cache.py:135: UpdatedStimulusPresentationTableWarning: 
	As of AllenSDK version 2.16.0, the latest Visual Behavior Ophys data has been significantly updated from previous releases. Specifically the user will need to update all processing of the stimulus_presentations tables. These tables now include multiple stimulus types delineated by the columns `stimulus_block` and `stimulus_block_name`.

The data that was available in previous releases are stored in the block name containing 'change_detection' and can be accessed in the pandas table by using: 
	`stimulus_presentations[stimulus_presentations.stimulus_block_name.str.contains('change_detection')]`
  warnings.warn(


In [6]:
all_ophys_experiments_table.head()

,behavior_session_id,ophys_session_id,ophys_container_id,mouse_id,indicator,full_genotype,driver_line,cre_line,reporter_line,sex,...,passive,experience_level,prior_exposures_to_session_type,prior_exposures_to_image_set,prior_exposures_to_omissions,date_of_acquisition,equipment_name,published_at,isi_experiment_id,file_id
ophys_experiment_id,,,,,,,,,,,,,,,,,,,,,
951980471,951520319,951410079,1018028342,457841,GCaMP6f,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,[Sst-IRES-Cre],Sst-IRES-Cre,Ai148(TIT2L-GC6f-ICL-tTA2),F,...,False,Familiar,0,65,0,2019-09-20 09:59:38.837000+00:00,MESO.1,2021-03-25,848974280,0
951980473,951520319,951410079,1018028345,457841,GCaMP6f,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,[Sst-IRES-Cre],Sst-IRES-Cre,Ai148(TIT2L-GC6f-ICL-tTA2),F,...,False,Familiar,0,65,0,2019-09-20 09:59:38.837000+00:00,MESO.1,2021-03-25,848974280,1
951980475,951520319,951410079,1018028339,457841,GCaMP6f,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,[Sst-IRES-Cre],Sst-IRES-Cre,Ai148(TIT2L-GC6f-ICL-tTA2),F,...,False,Familiar,0,65,0,2019-09-20 09:59:38.837000+00:00,MESO.1,2021-03-25,848974280,2
951980479,951520319,951410079,1018028354,457841,GCaMP6f,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,[Sst-IRES-Cre],Sst-IRES-Cre,Ai148(TIT2L-GC6f-ICL-tTA2),F,...,False,Familiar,0,65,0,2019-09-20 09:59:38.837000+00:00,MESO.1,2021-03-25,848974280,3
951980481,951520319,951410079,1018028357,457841,GCaMP6f,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,[Sst-IRES-Cre],Sst-IRES-Cre,Ai148(TIT2L-GC6f-ICL-tTA2),F,...,False,Familiar,0,65,0,2019-09-20 09:59:38.837000+00:00,MESO.1,2021-03-25,848974280,4


The filtering steps below can also be applied using this function: `visual_behavior.data_access.loading.get_platform_paper_experiment_table(add_extra_columns=True, limit_to_closest_active=True, include_4x2_data=False, remove_flagged=True, remove_Ai94=True)`
but are shown here for illustrative purposes

In [7]:
platform_experiments_table = all_ophys_experiments_table.copy()

# remove experiment with high variability in stimulus times 
problem_experiments = [856938751]
platform_experiments_table = platform_experiments_table.drop(problem_experiments, axis=0)

# Remove dataset with different experimental design (image set G/H & 4x2 multiscope configuration, 4 areas, 2 depths)
platform_experiments_table = platform_experiments_table[(platform_experiments_table.project_code != 'VisualBehaviorMultiscope4areasx2d')].copy()

# remove data with GCaMP6s (Ai94)
platform_experiments_table = platform_experiments_table[(platform_experiments_table.reporter_line != 'Ai94(TITL-GCaMP6s)')].copy()

# annotate table with useful information
platform_experiments_table = utilities.add_cell_type_column(platform_experiments_table)
platform_experiments_table = utilities.add_average_depth_across_container(platform_experiments_table)
platform_experiments_table = utilities.add_binned_depth_column(platform_experiments_table)
platform_experiments_table = utilities.add_area_depth_column(platform_experiments_table)
platform_experiments_table = utilities.add_layer_column(platform_experiments_table)
platform_experiments_table = utilities.add_area_layer_column(platform_experiments_table)

# add other columns indicating whether a session was the last familiar before the first novel session,
# or the second passing novel session after the first truly novel one
platform_experiments_table = utilities.add_date_string(platform_experiments_table)  # add simplified date string for sorting
platform_experiments_table = utilities.add_first_novel_column(platform_experiments_table)
platform_experiments_table = utilities.add_n_relative_to_first_novel_column(platform_experiments_table)
platform_experiments_table = utilities.add_last_familiar_column(platform_experiments_table)
platform_experiments_table = utilities.add_last_familiar_active_column(platform_experiments_table)
platform_experiments_table = utilities.add_second_novel_column(platform_experiments_table)
platform_experiments_table = utilities.add_second_novel_active_column(platform_experiments_table)
platform_experiments_table = utilities.add_experience_exposure_column(platform_experiments_table)

# filter table to only include the first novel session, the second novel session that is active behavior (not passive)
# and the last familiar session prior to the first novel session 
platform_experiments_table = utilities.limit_to_last_familiar_second_novel_active(platform_experiments_table)
platform_experiments_table = utilities.limit_to_containers_with_all_experience_levels(platform_experiments_table)

# rename experience level columns
platform_experiments_table['experience_level'] = [utils.convert_experience_level(experience_level) for experience_level in platform_experiments_table.experience_level.values]

In [8]:
# check the impact of filtering on number of experiments
print(len(all_ophys_experiments_table))
print(len(platform_experiments_table))

1936
402


In [10]:
platform_experiments_table.experience_level.unique()

array(['Familiar', 'Novel', 'Novel +'], dtype=object)

#### Get metadata for recorded cells

In [11]:
# Load full cell table from the cache
all_ophys_cells_table = cache.get_ophys_cells_table()

In [12]:
# inspect
all_ophys_cells_table.head()

,ophys_experiment_id,cell_specimen_id
cell_roi_id,,
1080884343,775614751,1086496928
1080884173,775614751,1086496914
1080883843,775614751,1086496838
1080886674,775614751,1086491756
1080885658,775614751,1086491699


In [13]:
# limit cells table to the set of experiments in the filtered experiments table
platform_cells_table = all_ophys_cells_table.reset_index().merge(platform_experiments_table, on='ophys_experiment_id')
platform_cells_table = platform_cells_table.set_index('cell_roi_id') # reset index

matched_cells_table = utilities.limit_to_cell_specimen_ids_matched_in_all_experience_levels(platform_cells_table)

In [14]:
# check the impact of filtering
print(len(all_ophys_cells_table.cell_specimen_id.unique()))
print(len(platform_cells_table.cell_specimen_id.unique()))
print(len(matched_cells_table.cell_specimen_id.unique()))

50476
14495
3921


In [15]:
# check that filtered cells table has the same experiments as the filtered experiments table
print(len(platform_experiments_table))
print(len(platform_cells_table.ophys_experiment_id.unique()))

402
402


In [16]:
# number of cells and experiments after matching across experience levels
print(len(matched_cells_table.ophys_experiment_id.unique()))
print(len(matched_cells_table.cell_specimen_id.unique()))

393
3921


It appears that 5 experiments that had all 3 experience levels (in `platform_experiments_table`) were dropped from the `matched_cells_table`, presumably because they did not contain any cells that were matched in all 3 experience levels

In [18]:
# which experiments are these? 
tmp = platform_experiments_table.reset_index()
tmp[tmp.ophys_experiment_id.isin(matched_cells_table.ophys_experiment_id)==False]

,ophys_experiment_id,behavior_session_id,ophys_session_id,ophys_container_id,mouse_id,indicator,full_genotype,driver_line,cre_line,reporter_line,...,layer,area_layer,date,first_novel,n_relative_to_first_novel,last_familiar,last_familiar_active,second_novel,second_novel_active,experience_exposure
255,920288855,920015895,919888953,1018028188,453990,GCaMP6f,Vip-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,[Vip-IRES-Cre],Vip-IRES-Cre,Ai148(TIT2L-GC6f-ICL-tTA2),...,lower,VISl_lower,20190808,False,-1.0,True,True,False,False,Familiar > 3
256,921024201,920839478,920695792,1018028188,453990,GCaMP6f,Vip-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,[Vip-IRES-Cre],Vip-IRES-Cre,Ai148(TIT2L-GC6f-ICL-tTA2),...,lower,VISl_lower,20190809,True,0.0,False,False,False,False,Novel 0
257,924198895,923834852,923705570,1018028188,453990,GCaMP6f,Vip-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,[Vip-IRES-Cre],Vip-IRES-Cre,Ai148(TIT2L-GC6f-ICL-tTA2),...,lower,VISl_lower,20190814,False,2.0,False,False,False,True,Novel 2
372,1080945465,1080558783,1080522184,1079027847,546605,GCaMP6f,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,[Sst-IRES-Cre],Sst-IRES-Cre,Ai148(TIT2L-GC6f-ICL-tTA2),...,lower,VISp_lower,20210202,False,-2.0,False,True,False,False,Familiar 0
373,1081870753,1081672735,1081652280,1079027847,546605,GCaMP6f,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,[Sst-IRES-Cre],Sst-IRES-Cre,Ai148(TIT2L-GC6f-ICL-tTA2),...,lower,VISp_lower,20210205,True,0.0,False,False,False,False,Novel 0
374,1082841182,1082707526,1082487233,1079027847,546605,GCaMP6f,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,[Sst-IRES-Cre],Sst-IRES-Cre,Ai148(TIT2L-GC6f-ICL-tTA2),...,lower,VISp_lower,20210209,False,1.0,False,False,True,True,Novel 2
375,1080945468,1080558783,1080522184,1079027857,546605,GCaMP6f,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,[Sst-IRES-Cre],Sst-IRES-Cre,Ai148(TIT2L-GC6f-ICL-tTA2),...,upper,VISl_upper,20210202,False,-2.0,False,True,False,False,Familiar 0
376,1081870756,1081672735,1081652280,1079027857,546605,GCaMP6f,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,[Sst-IRES-Cre],Sst-IRES-Cre,Ai148(TIT2L-GC6f-ICL-tTA2),...,upper,VISl_upper,20210205,True,0.0,False,False,False,False,Novel 0
377,1082841185,1082707526,1082487233,1079027857,546605,GCaMP6f,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,[Sst-IRES-Cre],Sst-IRES-Cre,Ai148(TIT2L-GC6f-ICL-tTA2),...,upper,VISl_upper,20210209,False,2.0,False,False,False,True,Novel 2


The experiments that were dropped were from inhibitory cre lines, which have few cells to begin with, and were from just 2 fields of view  (indicated by `ophys_container_ids`).

### Save annotated, filtered metadata tables

In [30]:
# save metadata tables
all_ophys_experiments_table.to_csv(os.path.join(platform_cache_dir, 'all_ophys_experiments_table.csv'))
platform_experiments_table.to_csv(os.path.join(platform_cache_dir, 'platform_paper_ophys_experiments_table.csv'))
all_ophys_cells_table.to_csv(os.path.join(platform_cache_dir, 'all_ophys_cells_table.csv'))
platform_cells_table.to_csv(os.path.join(platform_cache_dir, 'platform_paper_ophys_cells_table.csv'))
matched_cells_table.to_csv(os.path.join(platform_cache_dir, 'platform_paper_matched_ophys_cells_table.csv'))